In [2]:
import json
import numpy as np
import xarray as xr
import pandas as pd
from tqdm.auto import tqdm
from scipy.stats import uniform, norm
from itertools import combinations

from emulation import GP, loo
from icepy import volume_to_sle

2023-06-12 22:35:18.773241: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
class MultiGP:
    def __init__(self, X_data, Y_data, da_template=None):
        shape = Y_data.shape[1:]
        n_samples = len(Y_data)
        Y_data = Y_data.reshape(n_samples, -1)
        
        gps = np.empty(Y_data.shape[-1],dtype=object)
        for idx in range(Y_data.shape[-1]):
            gps[idx] = GP.from_data(
                X_data = X_data,
                Y_data = Y_data[:,idx].reshape(-1,1)
            )
            
        self.gps = gps.flatten()
        self.shape = shape
        self.da_template = da_template
        
    def train(self):
        for gp in tqdm(self.gps):
            gp.train()
            
    def predict(self, X):
        mean, var = np.zeros((len(X), len(self.gps))), np.zeros((len(X), len(self.gps)))
        for idx, gp in enumerate(self.gps):
            mean_temp, var_temp = gp.predict(X)
            mean[:,idx], var[:,idx] = mean_temp.flatten(), var_temp.flatten()
        mean, var = mean.reshape(len(X), *self.shape), var.reshape(len(X),*self.shape)
        if self.da_template is not None:
            mean_da, var_da = xr.ones_like(self.da_template), xr.ones_like(self.da_template)
            mean_da = mean_da.expand_dims(dim={"predict_idx": np.arange(len(X))}, axis=0) * mean
            var_da = var_da.expand_dims(dim={"predict_idx": np.arange(len(X))}, axis=0) * var
            return mean_da, var_da
        else:
            return mean, var
        

class MultiGPImp(MultiGP):
    def __init__(self, X_data, Y_data, metric_ds, da_template=None):
        super().__init__(X_data, Y_data, da_template=da_template)
        self.metric_ds = metric_ds
    
    def calc_imp(self, X, fudge_factor=1, return_full_imp=False):
        m_predict, m_var = self.predict(X)
        imp_numerator = (m_predict - self.metric_ds.metric_obs)**2
        imp_denominator = ((self.metric_ds.metric_sim_var + self.metric_ds.metric_obs_var)*fudge_factor + m_var)
        imp = np.sqrt(imp_numerator/imp_denominator)
        imp_mean = imp.mean(dim=["metric", "model", "time"])
        
        if return_full_imp is True:
            return imp_mean, imp
        else:
            return imp_mean

# PGM Metrics

In [4]:
# remove failed runs
pgm_ens_members = np.delete(np.arange(1000), 613)
pgm_ds = xr.open_dataset("../pd_ens/pgm_topo1.nc").sel(ensemble_idx=pgm_ens_members)

In [5]:
x, y = pgm_ds.x.values, pgm_ds.y.values
dA = np.abs(x[1] - x[0]) * np.abs(y[1] - y[0])
pgm_volume_da = pgm_ds.ice_thickness.sum(dim=["y", "x"]) * dA
pgm_volume_sle_da = volume_to_sle(pgm_volume_da)
pgm_max_thickness_da = pgm_ds.ice_thickness.max(dim=["y", "x"])

In [13]:
pgm_metrics_ds = xr.merge(
    [
        pgm_volume_da.rename("volume"),
        pgm_volume_sle_da.rename("volume_sle"),
        pgm_max_thickness_da.rename("max_thickness")
    ]
)
pgm_metrics_ds.to_netcdf("pgm_metrics1.nc")

# Load Data

In [14]:
ld_sample_df = pd.read_csv("../../data/param_sample/ld_lhs_values.csv")
ld_metrics_ds = xr.open_dataset("ld_metrics.nc")

# remove failed runs
pgm_ens_members = np.delete(np.arange(1000), 613)
pgm_sample_df = pd.read_csv("../pd_ens/sample.csv", index_col=0).iloc[pgm_ens_members]
# rename parameter to match ld
pgm_sample_df.rename({"g_pgm_ice_streams_ice_stream": "g_lgm_ice_streams_ice_stream"}, inplace=True, axis=1)
pgm_metrics_ds = xr.open_dataset("pgm_metrics1.nc").sel(ensemble_idx=pgm_ens_members)

In [ ]:
pgm_mod_ice_ds = xr.open_dataset("../pd_ens/pgm_topo0.nc").ice_thickness.sel(ensemble_idx = pgm_ens_members)
pgm_mod_vol = volume_to_sle((pgm_mod_ice_ds * 5000**2).sum(dim=["x", "y"]))

# Train Emulators

In [15]:
# implausibility emulator
imp_gp = MultiGPImp(
    X_data = ld_sample_df.values,
    Y_data = ld_metrics_ds.metric_sim_debiased.values,
    da_template = ld_metrics_ds.metric_sim.sel(ensemble_index=0),
    metric_ds = ld_metrics_ds,
)
imp_gp.train()

  0%|          | 0/24 [00:00<?, ?it/s]

In [ ]:
pgm_mod_vol_Y = pgm_mod_vol.values.reshape(-1,1)
pgm_mod_vol_gp = GP.from_data(
    X_data = pgm_sample_df.values,
    Y_data = pgm_mod_vol_Y
)
pgm_mod_vol_gp.train()

In [16]:
glac1d_20_vol_Y = volume_to_sle(ld_metrics_ds.metric_sim_debiased.sel(time=-20000, model="glac1d").sum(dim="metric").values).reshape(-1,1)
glac1d_20_vol_gp = GP.from_data(
    X_data = ld_sample_df.values,
    Y_data = glac1d_20_vol_Y
)
glac1d_20_vol_gp.train()

ice6g_22_vol_Y = volume_to_sle(ld_metrics_ds.metric_sim_debiased.sel(time=-22000, model="ice6g").sum(dim="metric").values).reshape(-1,1)
ice6g_22_vol_gp = GP.from_data(
    X_data = ld_sample_df.values,
    Y_data = ice6g_22_vol_Y
)
ice6g_22_vol_gp.train()

pgm_vol_Y = pgm_metrics_ds.volume_sle.values.reshape(-1,1)
pgm_vol_gp = GP.from_data(
    X_data = pgm_sample_df.values,
    Y_data = pgm_vol_Y
)
pgm_vol_gp.train()

pgm_max_thickness_Y = pgm_metrics_ds.max_thickness.values.reshape(-1,1)
pgm_max_thickness_gp = GP.from_data(
    X_data = pgm_sample_df.values,
    Y_data = pgm_max_thickness_Y
)
pgm_max_thickness_gp.train()

# Large Random Samples

In [18]:
# load last deglac dists
with open("../ld_ens/problem.json", "r") as f:
    ld_problem = json.load(f)
    
ld_problem['names'][0] = 'g_lgm_ice_streams_ice_stream'    
ld_ranges = {ld_problem['names'][idx]: ld_problem['bounds'][idx] for idx in range(len(ld_problem['names']))}
ld_dists = {name: uniform(loc=bound[0], scale=bound[1]-bound[0]) for name, bound in ld_ranges.items()}

# load pgm dists
with open("../pd_ens/problem.json", "r") as f:
    pgm_problem = json.load(f)

# modify to make margin parameter normal dist
pgm_problem['names'][0] = 'g_lgm_ice_streams_ice_stream'
pgm_problem['dists'] = pgm_problem['num_vars']*['unif']
margin_perc_idx = pgm_problem['names'].index('margin_perc')
pgm_problem['dists'][margin_perc_idx] = 'norm'
pgm_problem['bounds'][margin_perc_idx] = [0.5, 0.1]

pgm_ranges = {pgm_problem['names'][idx]: pgm_problem['bounds'][idx] for idx in range(len(pgm_problem['names']))}
pgm_dists = {name: uniform(loc=bound[0], scale=bound[1]-bound[0]) for name, bound in pgm_ranges.items()}
pgm_dists['margin_perc'] = norm(loc=0.5, scale=0.10)

In [192]:
N = int(1e5)
X_ld_rand = pd.DataFrame({name: dist.rvs(N) for name, dist in ld_dists.items()})
X_pgm_rand = pd.DataFrame({name: dist.rvs(N) for name, dist in pgm_dists.items()})
X_ld_rand.to_csv("ld_predict_sample.csv")
X_pgm_rand.to_csv("pgm_predict_sample.csv")

# Predict

In [233]:
pred_ld_regions_mean, pred_ld_regions_var = imp_gp.predict(
    X_ld_rand[ld_sample_df.columns].values,
)

In [237]:
pred_ld_regions_ds = xr.merge(
    [
        pred_ld_regions_mean.rename("metric_sim_mean"),
        pred_ld_regions_var.rename("metric_sim_var")
    ]
)
pred_ld_regions_ds.to_netcdf("ld_predict_regions.nc")

In [193]:
pred_ld_imp = imp_gp.calc_imp(
    X_ld_rand[ld_sample_df.columns].values,
    fudge_factor=1.2
)
pred_ld_nroy = pred_ld_imp < 3

pred_glac1d_20_vol, pred_glac1d_20_vol_var = glac1d_20_vol_gp.predict(X_ld_rand.values)
pred_ice6g_22_vol, pred_ice6g_22_vol_var = ice6g_22_vol_gp.predict(X_ld_rand.values)


In [194]:
coords = {"predict_index": np.arange(N)}
pred_ld_ds = xr.merge(
    [
        xr.DataArray(
            pred_glac1d_20_vol.flatten(), 
            coords=coords
        ).rename("glac1d_20_volume_mean"),
        xr.DataArray(
            pred_ice6g_22_vol.flatten(), 
            coords=coords
        ).rename("ice6g_22_volume_mean"),
        xr.DataArray(
            pred_glac1d_20_vol_var.flatten(), 
            coords=coords
        ).rename("glac1d_20_volume_var"),
        xr.DataArray(
            pred_ice6g_22_vol_var.flatten(), 
            coords=coords
        ).rename("ice6g_22_volume_var"),
        xr.DataArray(
            pred_ld_imp.values, 
            coords=coords
        ).rename("implausibility"),
        xr.DataArray(
            pred_ld_nroy.values, 
            coords=coords
        ).rename("nroy")
    ]
)
pred_ld_ds.to_netcdf("ld_predict.nc")


In [214]:
pred_pgm_sample_imp = imp_gp.calc_imp(
    pgm_sample_df[ld_sample_df.columns].values,
    fudge_factor=1.2
)
pred_pgm_sample_nroy = pred_pgm_sample_imp < 3

In [223]:
coords = {"ensemble_index": pgm_sample_df.index}
pgm_imp_ds = xr.merge(
    [
        xr.DataArray(
            pred_pgm_sample_imp.values.flatten(), 
            coords=coords
        ).rename("implausibility"),
        xr.DataArray(
            pred_pgm_sample_nroy.values.flatten(),
            coords=coords
        ).rename("nroy"),
    ]
)
pgm_imp_ds.to_netcdf("pgm_implausibility.nc")


In [195]:
pred_pgm_imp = imp_gp.calc_imp(
    X_pgm_rand[ld_sample_df.columns].values,
    fudge_factor=1.2
)
pred_pgm_nroy = pred_pgm_imp < 3

pred_pgm_vol, pred_pgm_vol_var = pgm_vol_gp.predict(X_pgm_rand.values)
pred_pgm_max_thickness, pred_pgm_max_thickness_var = pgm_max_thickness_gp.predict(X_pgm_rand.values)


In [196]:
# bias chosen since 20ka is largest simulated volume slice
# for glac and ice6g margins, we take the mean and scale
# by the mean ice volume from the large sample
ld_volume = ld_metrics_ds.metric_sim.sum(dim="metric").mean(dim="ensemble_index")
ld_volume_bias = ld_metrics_ds.metric_sim_bias.sum(dim="metric")
ld_volume_bias_perc = ld_volume_bias/ld_volume
mean_bias_perc = ld_volume_bias_perc.sel(time=-20000).mean().values

pgm_bias = pred_pgm_vol.mean() * mean_bias_perc

In [197]:
pred_pgm_vol_debiased = pred_pgm_vol - pgm_bias

In [198]:
coords = {"predict_index": np.arange(N)}
pred_pgm_ds = xr.merge(
    [
        xr.DataArray(
            pred_pgm_vol_debiased.flatten(), 
            coords=coords
        ).rename("volume_mean_debiased"),
        xr.DataArray(
            pgm_bias,
        ).rename("volume_bias"),
        xr.DataArray(
            mean_bias_perc,
        ).rename("volume_bias_perc"),
        xr.DataArray(
            pred_pgm_vol.flatten(), 
            coords=coords
        ).rename("volume_mean"),
        xr.DataArray(
            pred_pgm_vol_var.flatten(), 
            coords=coords
        ).rename("volume_var"),
        xr.DataArray(
            pred_pgm_max_thickness.flatten(), 
            coords=coords
        ).rename("max_thickness_mean"),
        xr.DataArray(
            pred_pgm_max_thickness_var.flatten(), 
            coords=coords
        ).rename("max_thickness_var"),
        xr.DataArray(
            pred_pgm_imp.values, 
            coords=coords
        ).rename("implausibility"),
        xr.DataArray(
            pred_pgm_nroy.values, 
            coords=coords
        ).rename("nroy")
    ]
)
pred_pgm_ds.to_netcdf("pgm_predict.nc")


In [229]:
pred_pgm_mod_vol, pred_pgm_mod_vol_var = pgm_mod_vol_gp.predict(X_pgm_rand.values)
pred_pgm_mod_vol_debiased = pred_pgm_mod_vol - pgm_bias

coords = {"predict_index": np.arange(N)}
pred_pgm_mod_ds = xr.merge(
    [
        xr.DataArray(
            pred_pgm_mod_vol_debiased.flatten(), 
            coords=coords
        ).rename("mod_volume_mean_debiased"),
        xr.DataArray(
            pred_pgm_mod_vol_var.flatten(), 
            coords=coords
        ).rename("mod_volume_var"),
    ]
)
pred_pgm_mod_ds.to_netcdf("pgm_mod_predict.nc")


# LD Optical Depth Panels

In [19]:
n_params = len(ld_ranges)
panel_res = 40
sample_res = 1000
param_combs = list(combinations(list(ld_ranges.keys()), 2))

In [ ]:
density_panels = np.zeros((len(param_combs),panel_res, panel_res))
mean_panels = np.zeros((len(param_combs),panel_res, panel_res))
min_panels = np.zeros((len(param_combs),panel_res, panel_res))
max_panels = np.zeros((len(param_combs),panel_res, panel_res))

for panel_idx, (p0_name, p1_name) in enumerate(tqdm(param_combs)):
    dists_subset = ld_dists.copy()
    dists_subset.pop(p0_name)
    dists_subset.pop(p1_name)

    p0 = np.linspace(*ld_ranges[p0_name], panel_res)
    p1 = np.linspace(*ld_ranges[p1_name], panel_res)

    panel = np.zeros((panel_res, panel_res, sample_res))

    for p0_idx in tqdm(range(panel_res), leave=False):
        for p1_idx in range(panel_res):

            panel_sample = pd.DataFrame({name: dist.rvs(sample_res) for name, dist in dists_subset.items()})
            panel_sample[p0_name] = p0[p0_idx]
            panel_sample[p1_name] = p1[p1_idx]
            imp_pred = imp_gp.calc_imp(
                panel_sample[list(ld_sample_df.columns)].values,
                fudge_factor=1.2
            )
            panel[p0_idx, p1_idx] = imp_pred.values
            
    density_panels[panel_idx] = np.sum(panel <= 3, axis=2)
    mean_panels[panel_idx] = np.mean(panel, axis=2)
    min_panels[panel_idx] = np.min(panel, axis=2)
    max_panels[panel_idx] = np.max(panel, axis=2)
    
density_panels = density_panels/sample_res

  0%|          | 0/21 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

In [ ]:
density_panels_2d = np.zeros((n_params, n_params, panel_res, panel_res))
mean_panels_2d = np.zeros((n_params, n_params, panel_res, panel_res))
min_panels_2d = np.zeros((n_params, n_params, panel_res, panel_res))
max_panels_2d = np.zeros((n_params, n_params, panel_res, panel_res))

for c_idx, comb in enumerate(param_combs):
    p0, p1 = comb
    p0_idx, p1_idx = list(ld_ranges.keys()).index(p0), list(ld_ranges.keys()).index(p1)
    density_panels_2d[p0_idx, p1_idx] = density_panels[c_idx].T
    density_panels_2d[p1_idx, p0_idx] = density_panels[c_idx]
    mean_panels_2d[p0_idx, p1_idx] = mean_panels[c_idx].T
    mean_panels_2d[p1_idx, p0_idx] = mean_panels[c_idx]
    min_panels_2d[p0_idx, p1_idx] = min_panels[c_idx].T
    min_panels_2d[p1_idx, p0_idx] = min_panels[c_idx]
    max_panels_2d[p0_idx, p1_idx] = max_panels[c_idx].T
    max_panels_2d[p1_idx, p0_idx] = max_panels[c_idx]



In [ ]:
panel_ds = xr.Dataset(
    data_vars = {
        "density": (["param_y", "param_x", "y", "x"], density_panels_2d),
        "mean": (["param_y", "param_x", "y", "x"], mean_panels_2d),
        "min": (["param_y", "param_x", "y", "x"], min_panels_2d),
        "max": (["param_y", "param_x", "y", "x"], max_panels_2d)
    },
    coords = {
        "param_y": list(ld_ranges.keys()),
        "param_x": list(ld_ranges.keys()),
    }
)
panel_ds.to_netcdf("ld_optical_panels1.nc")